<a href="https://colab.research.google.com/github/xmks-colab/SmartCutter/blob/main/SmartStitch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#@title Gerenciado de arquivos{ display-mode: "form" }
import os
import shutil
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript
import threading

class FileManager:
    print("Menu popup com: Apagar, Mover, Copiar, Renomear, Copiar caminho")
    def __init__(self, start_path='/content'):
        self.current_path = start_path
        self.output = widgets.Output()
        self.path_label = widgets.Label()
        self.status_label = widgets.Label()
        self.item_to_move = None
        self.item_to_copy = None
        self.multiselect_mode = False
        self.selected_items = set()
        self.refresh()

    def list_directory(self, path):
        try:
            items = os.listdir(path)
            items.sort()
            return items
        except Exception as e:
            return [f"Error: {e}"]

    def change_directory(self, folder):
        if self.multiselect_mode:
            return  # Não permite navegar em modo de multiseleção
        new_path = os.path.join(self.current_path, folder)
        if os.path.isdir(new_path):
            self.current_path = new_path
            self.refresh()

    def go_up(self):
        if self.multiselect_mode:
            return  # Não permite navegar em modo de multiseleção
        parent = os.path.dirname(self.current_path)
        if os.path.exists(parent):
            self.current_path = parent
            self.refresh()

    def toggle_multiselect(self):
        """Ativa/desativa o modo de multiseleção"""
        self.multiselect_mode = not self.multiselect_mode
        if not self.multiselect_mode:
            self.selected_items.clear()
        self.set_status(f"{'✅ Multiseleção ativada' if self.multiselect_mode else '❌ Multiseleção desativada'}")
        self.refresh()

    def select_all_items(self):
        """Seleciona todos os itens do diretório atual"""
        items = self.list_directory(self.current_path)
        # Remove mensagens de erro da lista
        items = [item for item in items if not item.startswith("Error:")]

        # Adiciona todos os itens à seleção
        for item in items:
            self.selected_items.add(item)

        self.set_status(f"✅ {len(items)} itens selecionados!")
        self.refresh()

    def toggle_item_selection(self, item):
        """Adiciona/remove item da seleção"""
        if item in self.selected_items:
            self.selected_items.remove(item)
        else:
            self.selected_items.add(item)
        self.refresh()

    def create_folder(self):
        folder_name_input = widgets.Text(
            placeholder='Digite o nome da pasta',
            description='Nome:',
            style={'description_width': 'initial'}
        )
        create_btn = widgets.Button(description='✅ Criar', button_style='success')
        cancel_btn = widgets.Button(description='❌ Cancelar')

        def do_create(btn):
            folder_name = folder_name_input.value.strip()
            if folder_name:
                new_folder_path = os.path.join(self.current_path, folder_name)
                try:
                    os.makedirs(new_folder_path, exist_ok=False)
                    self.set_status(f"✅ Pasta '{folder_name}' criada com sucesso!")
                    self.refresh()
                except FileExistsError:
                    self.set_status(f"❌ Pasta '{folder_name}' já existe!")
                except Exception as e:
                    self.set_status(f"❌ Erro ao criar pasta: {e}")
            else:
                self.set_status("❌ Nome da pasta não pode estar vazio!")

        def cancel_create(btn):
            self.refresh()

        with self.output:
            clear_output()
            self.path_label.value = f"📁 {self.current_path}"
            display(widgets.VBox([
                self.path_label,
                self.status_label,
                widgets.HTML("<h4>📁 Criar Nova Pasta</h4>"),
                folder_name_input,
                widgets.HBox([create_btn, cancel_btn])
            ]))

        create_btn.on_click(do_create)
        cancel_btn.on_click(cancel_create)

    def show_item_menu(self, item):
        """Mostra o menu popup com as opções para o item"""
        full_path = os.path.join(self.current_path, item)

        # Se estamos em modo de multiseleção, mostra menu diferente
        if self.multiselect_mode:
            self.show_multiselect_menu()
            return

        # Botões do menu normal
        delete_btn = widgets.Button(description='🗑️ Apagar', button_style='danger', layout=widgets.Layout(width='200px', margin='2px'))
        move_btn = widgets.Button(description='📦 Mover', layout=widgets.Layout(width='200px', margin='2px'))
        copy_btn = widgets.Button(description='📄 Copiar', button_style='warning', layout=widgets.Layout(width='200px', margin='2px'))
        rename_btn = widgets.Button(description='✏️ Renomear', button_style='primary', layout=widgets.Layout(width='200px', margin='2px'))
        copy_path_btn = widgets.Button(description='📋 Copiar Caminho', button_style='info', layout=widgets.Layout(width='200px', margin='2px'))
        multiselect_btn = widgets.Button(description='☑️ Multiseleção', button_style='info', layout=widgets.Layout(width='200px', margin='2px'))
        cancel_btn = widgets.Button(description='❌ Cancelar', layout=widgets.Layout(width='200px', margin='2px'))

        def on_delete(btn):
            self.delete_item(item)

        def on_move(btn):
            self.mark_for_move(item)

        def on_copy(btn):
            self.mark_for_copy(item)

        def on_rename(btn):
            self.rename_item(item)

        def on_copy_path(btn):
            js_code = f'''
            navigator.clipboard.writeText("{full_path}").then(function() {{
                console.log("Copiado com sucesso!");
            }}, function(err) {{
                console.error("Erro ao copiar: ", err);
            }});
            '''
            display(Javascript(js_code))
            self.set_status(f"📋 Caminho copiado: {os.path.basename(full_path)}")
            self.refresh()

        def on_multiselect(btn):
            self.toggle_multiselect()

        def on_cancel(btn):
            self.refresh()

        # Conectar eventos
        delete_btn.on_click(on_delete)
        move_btn.on_click(on_move)
        copy_btn.on_click(on_copy)
        rename_btn.on_click(on_rename)
        copy_path_btn.on_click(on_copy_path)
        multiselect_btn.on_click(on_multiselect)
        cancel_btn.on_click(on_cancel)

        # Mostrar menu popup
        with self.output:
            clear_output()
            self.path_label.value = f"📁 {self.current_path}"
            display(widgets.VBox([
                self.path_label,
                self.status_label,
                widgets.HTML(f"<h4>📋 Menu: {item}</h4>"),
                widgets.VBox([
                    delete_btn,
                    move_btn,
                    copy_btn,
                    rename_btn,
                    copy_path_btn,
                    multiselect_btn,
                    cancel_btn
                ], layout=widgets.Layout(border='1px solid #ddd', padding='10px', margin='10px'))
            ]))

    def show_multiselect_menu(self):
        """Mostra o menu para operações em multiseleção"""
        if not self.selected_items:
            self.set_status("❌ Nenhum item selecionado!")
            self.refresh()
            return

        # Botões do menu de multiseleção (sem renomear e copiar caminho)
        delete_btn = widgets.Button(description=f'🗑️ Apagar ({len(self.selected_items)})', button_style='danger', layout=widgets.Layout(width='250px', margin='2px'))
        move_btn = widgets.Button(description=f'📦 Mover ({len(self.selected_items)})', layout=widgets.Layout(width='250px', margin='2px'))
        copy_btn = widgets.Button(description=f'📄 Copiar ({len(self.selected_items)})', button_style='warning', layout=widgets.Layout(width='250px', margin='2px'))
        cancel_btn = widgets.Button(description='❌ Cancelar', layout=widgets.Layout(width='250px', margin='2px'))

        def on_multiselect_delete(btn):
            self.delete_multiple_items()

        def on_multiselect_move(btn):
            self.mark_multiple_for_move()

        def on_multiselect_copy(btn):
            self.mark_multiple_for_copy()

        def on_cancel(btn):
            self.refresh()

        # Conectar eventos
        delete_btn.on_click(on_multiselect_delete)
        move_btn.on_click(on_multiselect_move)
        copy_btn.on_click(on_multiselect_copy)
        cancel_btn.on_click(on_cancel)

        # Mostrar menu popup
        with self.output:
            clear_output()
            self.path_label.value = f"📁 {self.current_path}"
            selected_list = ", ".join(list(self.selected_items)[:3])
            if len(self.selected_items) > 3:
                selected_list += f" e mais {len(self.selected_items) - 3} itens"

            display(widgets.VBox([
                self.path_label,
                self.status_label,
                widgets.HTML(f"<h4>📋 Menu Multiseleção</h4>"),
                widgets.HTML(f"<p><strong>Selecionados:</strong> {selected_list}</p>"),
                widgets.VBox([
                    delete_btn,
                    move_btn,
                    copy_btn,
                    cancel_btn
                ], layout=widgets.Layout(border='1px solid #ddd', padding='10px', margin='10px'))
            ]))

    def delete_multiple_items(self):
        """Apaga múltiplos itens selecionados"""
        items_list = list(self.selected_items)
        confirm = widgets.Button(description=f'❗ Confirmar apagar {len(items_list)} itens', button_style='danger')
        cancel = widgets.Button(description='Cancelar')

        def do_delete_multiple(btn):
            success_count = 0
            error_count = 0
            for item in items_list:
                full_path = os.path.join(self.current_path, item)
                try:
                    if os.path.isdir(full_path):
                        shutil.rmtree(full_path)
                    else:
                        os.remove(full_path)
                    success_count += 1
                except Exception as e:
                    error_count += 1

            self.selected_items.clear()
            self.multiselect_mode = False
            self.set_status(f"✅ {success_count} itens apagados! {f'❌ {error_count} erros' if error_count > 0 else ''}")
            self.refresh()

        def cancel_delete(btn):
            self.refresh()

        with self.output:
            clear_output()
            self.path_label.value = f"📁 {self.current_path}"
            display(widgets.VBox([
                self.path_label,
                self.status_label,
                widgets.HTML(f"<h4>🗑️ Confirmar Exclusão Múltipla</h4>"),
                widgets.HTML(f"<p>Tem certeza que deseja apagar <strong>{len(items_list)} itens</strong>?</p>"),
                widgets.HTML(f"<p><small>{', '.join(items_list[:5])}{' ...' if len(items_list) > 5 else ''}</small></p>"),
                widgets.HBox([confirm, cancel])
            ]))

        confirm.on_click(do_delete_multiple)
        cancel.on_click(cancel_delete)

    def mark_multiple_for_move(self):
        """Marca múltiplos itens para mover"""
        # Cria lista com caminhos completos dos itens selecionados
        self.item_to_move = [os.path.join(self.current_path, item) for item in self.selected_items]
        self.item_to_copy = None
        self.multiselect_mode = False
        self.selected_items.clear()
        self.set_status(f"📦 {len(self.item_to_move)} itens selecionados para mover")
        self.refresh()

    def mark_multiple_for_copy(self):
        """Marca múltiplos itens para copiar"""
        # Cria lista com caminhos completos dos itens selecionados
        self.item_to_copy = [os.path.join(self.current_path, item) for item in self.selected_items]
        self.item_to_move = None
        self.multiselect_mode = False
        self.selected_items.clear()
        self.set_status(f"📄 {len(self.item_to_copy)} itens selecionados para copiar")
        self.refresh()

    def delete_item(self, item):
        full_path = os.path.join(self.current_path, item)
        confirm = widgets.Button(description=f'❗ Confirmar apagar: {item}', button_style='danger')
        cancel = widgets.Button(description='Cancelar')

        def do_delete(btn):
            try:
                if os.path.isdir(full_path):
                    shutil.rmtree(full_path)
                else:
                    os.remove(full_path)
                self.set_status(f"✅ {item} apagado com sucesso!")
                self.refresh()
            except Exception as e:
                self.set_status(f"❌ Erro ao apagar {item}: {e}")

        def cancel_delete(btn):
            self.refresh()

        with self.output:
            clear_output()
            self.path_label.value = f"📁 {self.current_path}"
            display(widgets.VBox([
                self.path_label,
                self.status_label,
                widgets.HTML(f"<h4>🗑️ Confirmar Exclusão</h4>"),
                widgets.HTML(f"<p>Tem certeza que deseja apagar: <strong>{item}</strong>?</p>"),
                widgets.HBox([confirm, cancel])
            ]))

        confirm.on_click(do_delete)
        cancel.on_click(cancel_delete)

    def rename_item(self, item):
        """Renomeia um arquivo ou pasta"""
        old_path = os.path.join(self.current_path, item)

        # Input para o novo nome
        new_name_input = widgets.Text(
            value=item,
            placeholder='Digite o novo nome',
            description='Novo nome:',
            style={'description_width': 'initial'}
        )
        confirm_btn = widgets.Button(description='✅ Renomear', button_style='primary')
        cancel_btn = widgets.Button(description='❌ Cancelar')

        def do_rename(btn):
            new_name = new_name_input.value.strip()
            if new_name and new_name != item:
                new_path = os.path.join(self.current_path, new_name)
                try:
                    os.rename(old_path, new_path)
                    self.set_status(f"✅ Renomeado de '{item}' para '{new_name}'!")
                    self.refresh()
                except FileExistsError:
                    self.set_status(f"❌ Já existe um item com o nome '{new_name}'!")
                except Exception as e:
                    self.set_status(f"❌ Erro ao renomear: {e}")
            elif not new_name:
                self.set_status("❌ Nome não pode estar vazio!")
            else:
                self.refresh()  # Nome não mudou, apenas voltar

        def cancel_rename(btn):
            self.refresh()

        with self.output:
            clear_output()
            self.path_label.value = f"📁 {self.current_path}"
            display(widgets.VBox([
                self.path_label,
                self.status_label,
                widgets.HTML(f"<h4>✏️ Renomear: {item}</h4>"),
                new_name_input,
                widgets.HBox([confirm_btn, cancel_btn])
            ]))

        confirm_btn.on_click(do_rename)
        cancel_btn.on_click(cancel_rename)

    def mark_for_move(self, item):
        self.item_to_move = os.path.join(self.current_path, item)
        self.item_to_copy = None  # Limpa seleção de cópia
        self.set_status(f"📦 Item selecionado para mover: {os.path.basename(item)}")
        self.refresh()

    def mark_for_copy(self, item):
        self.item_to_copy = os.path.join(self.current_path, item)
        self.item_to_move = None  # Limpa seleção de movimento
        self.set_status(f"📄 Item selecionado para copiar: {os.path.basename(item)}")
        self.refresh()

    def paste_item(self, btn=None):
        # Lidar com múltiplos itens para mover
        if isinstance(self.item_to_move, list):
            success_count = 0
            error_count = 0
            for src_path in self.item_to_move:
                dst = os.path.join(self.current_path, os.path.basename(src_path))
                try:
                    if src_path != dst:  # Só move se não for a mesma localização
                        shutil.move(src_path, dst)
                        success_count += 1
                    else:
                        # Se for o mesmo local, considera como sucesso (não precisa mover)
                        success_count += 1
                except Exception as e:
                    error_count += 1
                    print(f"Erro ao mover {src_path}: {e}")  # Debug

            self.item_to_move = None
            self.set_status(f"✅ {success_count} itens movidos! {f'❌ {error_count} erros' if error_count > 0 else ''}")
            self.refresh()
            return

        # Lidar com múltiplos itens para copiar
        if isinstance(self.item_to_copy, list):
            success_count = 0
            error_count = 0
            for src_path in self.item_to_copy:
                dst = os.path.join(self.current_path, os.path.basename(src_path))
                try:
                    if src_path != dst:  # Só copia se não for a mesma localização
                        if os.path.isdir(src_path):
                            shutil.copytree(src_path, dst)
                        else:
                            shutil.copy2(src_path, dst)
                        success_count += 1
                    else:
                        # Se for o mesmo local, cria uma cópia com nome diferente
                        base_name = os.path.basename(src_path)
                        name, ext = os.path.splitext(base_name)
                        counter = 1
                        while os.path.exists(dst):
                            if ext:
                                new_name = f"{name}_copia{counter}{ext}"
                            else:
                                new_name = f"{name}_copia{counter}"
                            dst = os.path.join(self.current_path, new_name)
                            counter += 1

                        if os.path.isdir(src_path):
                            shutil.copytree(src_path, dst)
                        else:
                            shutil.copy2(src_path, dst)
                        success_count += 1
                except Exception as e:
                    error_count += 1
                    print(f"Erro ao copiar {src_path}: {e}")  # Debug

            self.item_to_copy = None
            self.set_status(f"✅ {success_count} itens copiados! {f'❌ {error_count} erros' if error_count > 0 else ''}")
            self.refresh()
            return

        # Lógica original para item único
        if self.item_to_move:
            src = self.item_to_move
            dst = os.path.join(self.current_path, os.path.basename(src))
            try:
                self.set_status("⏳ Movendo...")
                shutil.move(src, dst)
                self.item_to_move = None
                self.set_status("✅ Movido com sucesso!")
            except Exception as e:
                self.set_status(f"❌ Erro ao mover: {e}")
            self.refresh()

        elif self.item_to_copy:
            src = self.item_to_copy
            dst = os.path.join(self.current_path, os.path.basename(src))
            try:
                self.set_status("⏳ Copiando...")
                if os.path.isdir(src):
                    shutil.copytree(src, dst)
                else:
                    shutil.copy2(src, dst)
                self.item_to_copy = None
                self.set_status("✅ Copiado com sucesso!")
            except Exception as e:
                self.set_status(f"❌ Erro ao copiar: {e}")
            self.refresh()

    def set_status(self, msg, duration=3):
        self.status_label.value = msg
        if duration > 0:
            threading.Timer(duration, self.clear_status).start()

    def clear_status(self):
        self.status_label.value = ""

    def refresh(self):
        with self.output:
            clear_output()

            # Mostra se está em modo de multiseleção
            path_text = f"📁 {self.current_path}"
            if self.multiselect_mode:
                path_text += f" | ☑️ Multiseleção ({len(self.selected_items)} selecionados)"

            self.path_label.value = path_text

            display(widgets.VBox([
                self.path_label,
                self.status_label
            ]))

            rows = []

            # Botão para voltar (desabilitado em multiseleção)
            if self.current_path != '/' and not self.multiselect_mode:
                up_button = widgets.Button(description='🔼 ..', layout=widgets.Layout(width='auto'))
                up_button.on_click(lambda btn: self.go_up())
                rows.append(widgets.HBox([up_button]))

            # Botões de controle
            control_buttons = []

            # Botão para criar pasta (desabilitado em multiseleção)
            if not self.multiselect_mode:
                create_folder_btn = widgets.Button(description='📁➕ Nova Pasta', button_style='info', layout=widgets.Layout(width='auto'))
                create_folder_btn.on_click(lambda btn: self.create_folder())
                control_buttons.append(create_folder_btn)

            # Botões do modo multiseleção
            if self.multiselect_mode:
                # Botão para selecionar tudo
                select_all_btn = widgets.Button(description='☑️ Selecionar Tudo', button_style='success', layout=widgets.Layout(width='auto'))
                select_all_btn.on_click(lambda btn: self.select_all_items())
                control_buttons.append(select_all_btn)

                # Botão para sair do modo multiseleção
                exit_multiselect_btn = widgets.Button(description='❌ Sair da Multiseleção', button_style='warning', layout=widgets.Layout(width='auto'))
                exit_multiselect_btn.on_click(lambda btn: self.toggle_multiselect())
                control_buttons.append(exit_multiselect_btn)

            if control_buttons:
                rows.append(widgets.HBox(control_buttons))

            # Botão para colar item (se houver)
            if isinstance(self.item_to_move, list) and self.item_to_move:
                paste_btn = widgets.Button(description=f'📥 Colar aqui (mover {len(self.item_to_move)} itens)', button_style='success')
                paste_btn.on_click(self.paste_item)
                rows.append(widgets.HBox([paste_btn]))
            elif self.item_to_move and not isinstance(self.item_to_move, list):
                paste_btn = widgets.Button(description=f'📥 Colar aqui (mover): {os.path.basename(self.item_to_move)}', button_style='success')
                paste_btn.on_click(self.paste_item)
                rows.append(widgets.HBox([paste_btn]))

            if isinstance(self.item_to_copy, list) and self.item_to_copy:
                paste_btn = widgets.Button(description=f'📥 Colar aqui (copiar {len(self.item_to_copy)} itens)', button_style='warning')
                paste_btn.on_click(self.paste_item)
                rows.append(widgets.HBox([paste_btn]))
            elif self.item_to_copy and not isinstance(self.item_to_copy, list):
                paste_btn = widgets.Button(description=f'📥 Colar aqui (copiar): {os.path.basename(self.item_to_copy)}', button_style='warning')
                paste_btn.on_click(self.paste_item)
                rows.append(widgets.HBox([paste_btn]))

            # Lista de itens do diretório
            items = self.list_directory(self.current_path)
            for item in items:
                full_path = os.path.join(self.current_path, item)
                is_dir = os.path.isdir(full_path)

                # Em modo multiseleção, mostra checkbox
                if self.multiselect_mode:
                    is_selected = item in self.selected_items
                    checkbox_text = "☑️" if is_selected else "☐"
                    label = f"{checkbox_text} {'📁' if is_dir else '📄'} {item}"

                    select_btn = widgets.Button(
                        description=label,
                        layout=widgets.Layout(width='80%'),
                        button_style='info' if is_selected else ''
                    )
                    select_btn.on_click(lambda btn, name=item: self.toggle_item_selection(name))

                    menu_btn = widgets.Button(description='...', layout=widgets.Layout(width='15%'), button_style='')
                    menu_btn.on_click(lambda btn, name=item: self.show_item_menu(name))

                    rows.append(widgets.HBox([select_btn, menu_btn]))

                else:
                    # Modo normal
                    label = f"📁 {item}" if is_dir else f"📄 {item}"

                    open_btn = widgets.Button(description=label, layout=widgets.Layout(width='80%'))
                    menu_btn = widgets.Button(description='...', layout=widgets.Layout(width='15%'), button_style='')

                    # Função para abrir pastas
                    if is_dir:
                        open_btn.on_click(lambda btn, folder=item: self.change_directory(folder))

                    # Função para mostrar menu
                    menu_btn.on_click(lambda btn, name=item: self.show_item_menu(name))

                    rows.append(widgets.HBox([open_btn, menu_btn]))

            display(widgets.VBox(rows))

    def show(self):
        display(self.output)


fm = FileManager()
fm.show()

In [ ]:

#@title SmartStitch { display-mode: "form" }
import ipywidgets as widgets
from IPython.display import display
import subprocess
import os
from google.colab import files
import shutil
import sys

def run_command(cmd, silent=True):
    """Executa comando silenciosamente"""
    try:
        if silent:
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True, check=True)
        else:
            result = subprocess.run(cmd, shell=True, check=True)
        return True, result
    except subprocess.CalledProcessError as e:
        return False, e

def install_smartstitch():
    print("Preparando...")

    # Verifica se o diretório já existe
    if os.path.exists('SmartStitch'):
        print("SmartStitch já está instalado. Ignorando...")
        os.chdir('SmartStitch')
        print("Finalizado.")
        return

    # Clone do repositório
    success, _ = run_command("git clone https://github.com/MechTechnology/SmartStitch.git")
    if not success:
        print("Erro ao clonar o repositório.")
        return

    # Mudança para o diretório
    os.chdir('SmartStitch')

    # Instalação das dependências
    packages = ["numpy", "pillow", "natsort", "psd-tools"]
    for package in packages:
        success, _ = run_command(f"pip install {package}")
        if not success:
            print(f"Erro ao instalar {package}.")
            return

    print("Finalizado.")

# Executar instalação
install_smartstitch()

# --- Criar pasta input automaticamente ---
def create_input_folder():
    input_path = "/content/SmartStitch/input"
    os.makedirs(input_path, exist_ok=True)
    return input_path

# --- pasta imutavel ---
input_folder = "input"

split_height = widgets.IntText(
    value=5000,
    description="Split H:",
    layout=widgets.Layout(width="40%")
)

output_format = widgets.Dropdown(
    options=[".png", ".jpg", ".webp", ".bmp", ".psd", ".tiff", ".tga"],
    value=".png",
    description="Formato:"
)

custom_width = widgets.IntText(
    value=0,
    description="Largura:",
    layout=widgets.Layout(width="30%")
)

detection_type = widgets.Dropdown(
    options=["pixel", "none"],
    value="pixel",
    description="Detecção:"
)

sensitivity = widgets.IntSlider(
    value=90, min=0, max=100, step=1,
    description="Sensib:"
)

quality = widgets.IntSlider(
    value=100, min=1, max=100, step=1,
    description="Qualidade:"
)

ignorable_pixels = widgets.IntText(
    value=5,
    description="Ign Pixels:"
)

scan_line = widgets.IntSlider(
    value=5, min=1, max=100, step=1,
    description="Scan Line:"
)

# --- Checkbox para zip e download ---
zip_download = widgets.Checkbox(
    value=True,
    description="Zipar e Baixar",
    indent=False
)

# --- 2. Saída e Função de Upload ---
output = widgets.Output()

def upload_images(b):
    output.clear_output()
    with output:
        input_path = '/content/SmartStitch/input'
        print(f"📁 Usando pasta input: {input_path}")

        print("📤 Selecione as imagens para upload...")

        original_cwd = os.getcwd()
        try:
            os.chdir(input_path)
            uploaded = files.upload()
        finally:
            os.chdir(original_cwd)

        if uploaded:
            print(f"📥 {len(uploaded)} arquivo(s) salvo(s) diretamente na pasta input...")
            for filename in uploaded.keys():
                dst_path = f"{input_path}/{filename}"
                print(f"✅ {filename} → {dst_path}")

            print(f"🎉 Upload concluído! {len(uploaded)} imagem(s) salvas em {input_path}")
        else:
            print("❌ Nenhum arquivo foi selecionado")

def run_smartstitch(b):
    output.clear_output()

    # Garantir que a pasta existe antes de executar
    input_path = create_input_folder()

    with output:
        # Informação sobre imagens (opcional, apenas para informar o usuário)
        if os.path.exists(input_path):
            images = [f for f in os.listdir(input_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.webp', '.tiff', '.tga', '.psd'))]
            if images:
                print(f"📸 Encontradas {len(images)} imagem(s): {', '.join(images[:3])}{'...' if len(images) > 3 else ''}")

    cmd = [
        "python", "SmartStitchConsole.py",
        "-i", input_folder,
        "-sh", str(split_height.value),
        "-t", output_format.value
    ]

    if custom_width.value > 0:
        cmd += ["-cw", str(custom_width.value)]
    if detection_type.value:
        cmd += ["-dt", detection_type.value]
    if sensitivity.value:
        cmd += ["-s", str(sensitivity.value)]
    if quality.value:
        cmd += ["-lq", str(quality.value)]
    if ignorable_pixels.value:
        cmd += ["-ip", str(ignorable_pixels.value)]
    if scan_line.value:
        cmd += ["-sl", str(scan_line.value)]

    with output:
        print(f"\n🚀 Executando: {' '.join(cmd)}")
        print("-" * 50)
        process = subprocess.run(cmd, text=True, capture_output=True)

        if process.stdout.strip():
            print("📋 Output:")
            print(process.stdout)

        if process.stderr.strip():
            print("⚠️ Avisos/Erros:")
            print(process.stderr)

        print("-" * 50)
        print("✅ Processamento concluído!")

        # Zip e download se checkbox ativado
        if zip_download.value:
            output_folder = "/content/SmartStitch/input [stitched]"
            if os.path.exists(output_folder):
                zip_path = "/content/SmartStitch/stitched_results"
                shutil.make_archive(zip_path, 'zip', output_folder)
                zip_file = f"{zip_path}.zip"
                print(f"📦 Zip criado: {zip_file}")
                print("⬇️ Iniciando download...")
                files.download(zip_file)
            else:
                print(f"❌ Pasta de saída não encontrada: {output_folder}")

def clear_input_folder(b):
    output.clear_output()
    input_path = '/content/SmartStitch/input'
    with output:
        if os.path.exists(input_path):
            for filename in os.listdir(input_path):
                file_path = os.path.join(input_path, filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                except Exception as e:
                    print(f"❌ Erro ao limpar {file_path}: {e}")
            print("✅ Pasta de entrada limpa!")
        else:
            print("❌ Pasta de entrada não existe!")

def clear_output_folder(b):
    output.clear_output()
    output_folder = '/content/SmartStitch/input [stitched]'
    with output:
        if os.path.exists(output_folder):
            shutil.rmtree(output_folder)
            print("✅ Pasta de saída limpa!")
        else:
            print("❌ Pasta de saída não existe!")

# --- 3. Botões ---
upload_button = widgets.Button(
    description="📤 Upload",
    button_style="info",
    tooltip="Enviar imagens para a pasta input"
)
upload_button.on_click(upload_images)

run_button = widgets.Button(
    description="▶️ Executar",
    button_style="success",
    tooltip="Executar SmartStitch com as configurações atuais"
)
run_button.on_click(run_smartstitch)

clear_input_button = widgets.Button(
    description="Lpar Entrada",
    button_style="danger",
    tooltip="Limpar a pasta de entrada"
)
clear_input_button.on_click(clear_input_folder)

clear_output_button = widgets.Button(
    description="Lpar Saida",
    button_style="danger",
    tooltip="Limpar a pasta de saida"
)
clear_output_button.on_click(clear_output_folder)

# --- 4. Montar Layout ---
ui = widgets.VBox([
    widgets.HTML("<h3>🧵 SmartStitch Console GUI</h3>"),
    widgets.HTML("<h4>⚙️ Configurações</h4>"),
    widgets.HBox([split_height, custom_width]),
    widgets.HBox([output_format, detection_type]),

    widgets.HTML("<h4>🎚️ Parâmetros Avançados</h4>"),
    sensitivity,
    quality,
    ignorable_pixels,
    scan_line,

    widgets.HTML("<h4>🎬 Execução</h4>"),
    widgets.HBox([upload_button, run_button, clear_input_button, clear_output_button]),
    zip_download,

    widgets.HTML("<h4>📊 Output</h4>"),
    output
])

# Criar a pasta input automaticamente ao inicializar
print("🔧 Inicializando SmartStitch GUI...")
create_input_folder()
#print("✅ Pasta input criada em /content/SmartStitch/input")

display(ui)